In [1]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
import torch
from torch_geometric.utils.convert import from_networkx
from pygod.detector import DOMINANT, OCGNN, GUIDE, GAE, GAAN, AnomalyDAE, CONAD
from pygod.metric import eval_average_precision, eval_roc_auc, eval_f1, eval_precision_at_k, eval_recall_at_k
from pygod.generator import gen_contextual_outlier, gen_structural_outlier
import pickle
import time
device = torch.device('cuda')

In [3]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

# Load Dataset

In [66]:
unsw_labeled_path = "C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\unsw_labeled.parquet"

In [67]:
unsw = pd.read_parquet(unsw_labeled_path)

In [68]:
unsw.info()
unsw.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 125180 entries, 1 to 490022
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   source_ip         125180 non-null  object  
 1   destination_ip    125180 non-null  object  
 2   source_port       125180 non-null  object  
 3   destination_port  125180 non-null  object  
 4   info_message      125180 non-null  object  
 5   attack_category   15657 non-null   category
 6   is_malware        125180 non-null  int64   
 7   source_ip_info    125180 non-null  object  
 8   source_port_info  125180 non-null  object  
 9   dest_ip_info      125180 non-null  object  
 10  dest_port_info    125180 non-null  object  
 11  count_benign      125180 non-null  int64   
 12  count_malware     125180 non-null  int64   
dtypes: category(1), int64(3), object(9)
memory usage: 12.5+ MB


,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info,count_benign,count_malware
index,,,,,,,,,,,,,
1,175.45.176.1,149.171.126.18,4657,80,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1,149.171.126.18 GET /oKmwKoVbq HTTP/1.1,80 GET /oKmwKoVbq HTTP/1.1,1,0
2,175.45.176.3,149.171.126.18,32473,80,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,1,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...,149.171.126.18 GET /level/15/exec/-/buffers/as...,80 GET /level/15/exec/-/buffers/assigned/dump ...,1,7
6,175.45.176.0,149.171.126.17,49194,80,GET eLWfxXSPkc HTTP/1.1,NaN,0,175.45.176.0 GET eLWfxXSPkc HTTP/1.1,49194 GET eLWfxXSPkc HTTP/1.1,149.171.126.17 GET eLWfxXSPkc HTTP/1.1,80 GET eLWfxXSPkc HTTP/1.1,1,0


# Function

In [69]:
def split_train_test(df, test_size=0.3):
    train, test = train_test_split(df, test_size=test_size)
    return train, test

In [70]:
def graph_modeling_1(df):
    graph = nx.Graph()
    node_features = []
    labels = []

    for source_port_info in df["source_port_info"].unique():
        graph.add_node(source_port_info)
        info_message = df[df["source_port_info"] == source_port_info]["info_message"].iloc[0]
        label = df[df["source_port_info"] == source_port_info]["is_malware"].iloc[0]
        node_features.append([float(len(info_message))])
        labels.append(label)
        
    for (source_ip), group in df.groupby(["source_ip"]):
        for i in range(len(group) - 1):
            from_node = group.iloc[i]["source_port_info"]
            to_node = group.iloc[i+1]["source_port_info"]
            if graph.has_edge(from_node, to_node):
                graph[from_node][to_node]["weight"] += 1
            else:
                graph.add_edge(from_node, to_node, weight=1)
    return graph, node_features, labels

In [71]:
def graph_modeling_2(df):
    graph = nx.Graph()
    node_features = []
    labels = []

    for info_message in df["info_message"].unique():
        graph.add_node(info_message)
        node_features.append([float(len(info_message))])
    
    for (source_ip), group in df.groupby(["source_ip"]):
        for i in range(len(group) - 1):
            from_node = group.iloc[i]["info_message"]
            to_node = group.iloc[i+1]["info_message"]
            if graph.has_edge(from_node, to_node):
                graph[from_node][to_node]["weight"] += 1
            else:
                graph.add_edge(from_node, to_node, weight=1)
    node_features = torch.tensor(node_features)
    labels = torch.tensor(labels)
    return graph, node_features, labels

# Eksperimen 1

In [56]:
unsw['source_port'] = unsw.source_port.astype('int32')
unsw['destination_port']= unsw.destination_port.astype('int32')

In [57]:
train_df, test_df = split_train_test(unsw)

In [58]:
train_df.is_malware.value_counts()

is_malware
0    77233
1    10393
Name: count, dtype: int64

In [59]:
nD_train_df = train_df.drop_duplicates(subset=['info_message'], keep='first')
label_train = nD_train_df['is_malware'].to_numpy()
label_train = torch.tensor(label_train, dtype=torch.float)
value_counts = np.unique(label_train, return_counts=True)
print(value_counts)

(array([0., 1.], dtype=float32), array([1653, 5923], dtype=int64))


In [60]:
nD_train_df.is_malware.value_counts()

is_malware
1    5923
0    1653
Name: count, dtype: int64

In [69]:
test_df.is_malware.value_counts()

is_malware
0    33041
1      863
Name: count, dtype: int64

In [62]:
nD_test_df = test_df.drop_duplicates(subset=['info_message'], keep='first')
nD_test_df.is_malware.value_counts()
label_test = nD_test_df['is_malware'].to_numpy()
label_test = torch.tensor(label_test, dtype=torch.float)
value_counts = np.unique(label_test, return_counts=True)
print(value_counts)

(array([0., 1.], dtype=float32), array([ 821, 3060], dtype=int64))


In [67]:
train_graph, train_node_features = graph_modeling_2(train_df)

In [68]:
# number of nodes
print(train_graph.number_of_nodes())

7576


In [70]:
test_graph, test_node_features = graph_modeling_2(test_df)

### DOMINANT

In [75]:
pyG_train = from_networkx(train_graph)
pyG_train.x = train_node_features
pyG_test = from_networkx(test_graph)
pyG_test.x = test_node_features

In [78]:
dominant_model = DOMINANT(gpu=0, weight=0.02, num_layers=64, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)

In [79]:
dominant_compile = dominant_model.fit(pyG_train, label_train)

Epoch 0000: Loss 4.8750 | AUC 0.6216 | Recall 0.7873 | Precision 0.7873 | AP 0.8504 | F1 0.7783 | Time 12.57
Epoch 0001: Loss 4.8739 | AUC 0.6213 | Recall 0.7874 | Precision 0.7874 | AP 0.8514 | F1 0.7874 | Time 12.25
Epoch 0002: Loss 4.8740 | AUC 0.6214 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.20
Epoch 0003: Loss 4.8726 | AUC 0.6213 | Recall 0.7874 | Precision 0.7874 | AP 0.8514 | F1 0.7874 | Time 12.20
Epoch 0004: Loss 4.8725 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.24
Epoch 0005: Loss 4.8718 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.26
Epoch 0006: Loss 4.8707 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.07
Epoch 0007: Loss 4.8701 | AUC 0.6214 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 11.98
Epoch 0008: Loss 4.8685 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.03
Epoch 0009: Loss 4.

In [80]:
dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)

Test: Loss 0.0021 | 

ValueError: Found input variables with inconsistent numbers of samples: [3881, 1582]

In [ ]:
f1_score_ip = eval_f1(label_test, dominant_ip_pred_res)
print(f1_score_ip)
precision = eval_precision_at_k(label_test, dominant_ip_score_res, k=1606)
print(precision)
recall = eval_recall_at_k(label_test, dominant_ip_score_res, k=1606)
print(recall)

In [73]:
def dominant(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test):
    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    pyG_test = from_networkx(test_graph)
    pyG_test.x = test_node_features

    model = DOMINANT(gpu=0, weight=0.02, num_layers=64, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)
    dominant_compile = model.fit(pyG_train)
    dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
    f1_score_ip = eval_f1(label_test, dominant_ip_pred_res)
    precision = eval_precision_at_k(label_test, dominant_ip_score_res, k=3881)
    recall = eval_recall_at_k(label_test, dominant_ip_score_res, k=3881)
    return f1_score_ip, precision, recall

In [74]:
f1 = []
precision = []
recall = []
for i in range(3):
    f1_score, precision_score, recall_score = dominant(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test)
    f1.append(f1_score)
    precision.append(precision_score)
    recall.append(recall_score)

Epoch 0000: Loss 4.8750 |  | Time 12.97
Epoch 0001: Loss 4.8732 |  | Time 12.00
Epoch 0002: Loss 4.8745 |  | Time 12.29


KeyboardInterrupt: 

# Eksperimen 2

In [72]:
train_df, test_df = split_train_test(unsw)

In [32]:
nD_train_df = train_df.drop_duplicates(subset=['source_port_info'], keep='first')

In [144]:
nD_train_df.is_malware.value_counts()

is_malware
0    10070
1     5381
Name: count, dtype: int64

In [140]:
value_counts = train_df.is_malware.value_counts()
if value_counts[0] > 10398:
    benign = train_df[train_df['is_malware'] == 0].sample(n=10398)
    malicious = train_df[train_df['is_malware'] == 1].sample(n=5699)
    train_df = pd.concat([benign, malicious])
    # nD_train_df = train_df.copy()

In [141]:
train_df.is_malware.value_counts()

is_malware
0    10398
1     5699
Name: count, dtype: int64

In [33]:
test_df.is_malware.value_counts()

is_malware
0    33097
1     4457
Name: count, dtype: int64

In [34]:
value_counts = test_df.is_malware.value_counts()
if value_counts[0] > 5321:
    df_to_lower = test_df[test_df['is_malware'] == 0].sample(n=5321)
    test_df = pd.concat([test_df[test_df['is_malware'] == 1], df_to_lower])
    # nD_test_df = test_df.copy()

In [147]:
nD_test_df = test_df.drop_duplicates(subset=['source_port_info'], keep='first')
nD_test_df.is_malware.value_counts()

is_malware
0    5229
1    4296
Name: count, dtype: int64

In [73]:
train_graph, train_node_features, label_train = graph_modeling_1(train_df)

In [74]:
test_graph, test_node_features, label_test = graph_modeling_1(test_df)

In [9]:
pickle.dump(train_graph, open('model_graph/train_graph.pkl', 'wb'))
pickle.dump(train_node_features, open('model_graph/train_node_features.pkl', 'wb'))
pickle.dump(label_train, open('model_graph/label_train.pkl', 'wb'))

NameError: name 'train_graph' is not defined

In [152]:
pickle.dump(test_graph, open('model_graph/test_graph.pkl', 'wb'))
pickle.dump(test_node_features, open('model_graph/test_node_features.pkl', 'wb'))
pickle.dump(label_test, open('model_graph/label_test.pkl', 'wb'))

In [4]:
train_graph = pickle.load(open('model_graph/train_graph.pkl', 'rb'))
label_train = pickle.load(open('model_graph/label_train.pkl', 'rb'))
train_node_features = pickle.load(open('model_graph/train_node_features.pkl', 'rb'))

In [5]:
test_graph = pickle.load(open('model_graph/test_graph.pkl', 'rb'))
label_test = pickle.load(open('model_graph/label_test.pkl', 'rb'))
test_node_features = pickle.load(open('model_graph/test_node_features.pkl', 'rb'))

In [6]:
train_graph.number_of_nodes()

15451

In [7]:
label_train

tensor([0, 0, 0,  ..., 1, 1, 1])

In [8]:
unique_values, counts = torch.unique(label_test, return_counts=True)
print(unique_values, counts)

tensor([0, 1]) tensor([5229, 4296])


In [9]:
test_graph.number_of_nodes()

9525

In [10]:
test_node_features

tensor([[22.],
        [34.],
        [27.],
        ...,
        [28.],
        [15.],
        [28.]])

In [11]:
import torchvision.models as models
import torch_geometric.nn as pyg_nn

## DOMINANT

In [103]:
def make_dominant_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)
    test_node_features = torch.tensor(test_node_features)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    dominant_model = DOMINANT(gpu=0, weight=0.1, num_layers=8, hid_dim=64, contamination=0.37, lr=0.001, verbose=3, backbone=pyg_nn.EdgeCNN, epoch=100)  
    dominant_compile = dominant_model.fit(pyG_train)
    return dominant_compile, pyG_test

In [104]:
def predict_dominant(label_test, dominant_compile, pyG_test):
    
    label_test = torch.tensor(label_test)
    dominant_ip_pred_res, dominant_ip_score_res = dominant_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    
    unique_values, counts = torch.unique(dominant_ip_pred_res, return_counts=True)
    print(unique_values, counts)

    f1_pygod = eval_f1(label_test, dominant_ip_pred_res)
    precision_pygod = eval_precision_at_k(label_test, dominant_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, dominant_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return precision_pygod, recall_pygod, f1_pygod

In [105]:
dominant_model, graph_test = make_dominant_model(train_graph, train_node_features, label_train, test_graph, test_node_features)
precision_score, recall_score, f1_score = predict_dominant(label_test, dominant_model, graph_test)

Epoch 0000: Loss 58.7736 |  | Time 0.85
Epoch 0001: Loss 54.3093 |  | Time 0.98
Epoch 0002: Loss 49.9715 |  | Time 0.97
Epoch 0003: Loss 46.1873 |  | Time 0.97
Epoch 0004: Loss 42.1571 |  | Time 0.97
Epoch 0005: Loss 38.6776 |  | Time 0.97
Epoch 0006: Loss 36.1402 |  | Time 0.97
Epoch 0007: Loss 32.9313 |  | Time 0.97
Epoch 0008: Loss 30.5166 |  | Time 0.97
Epoch 0009: Loss 28.3955 |  | Time 1.00
Epoch 0010: Loss 26.4588 |  | Time 0.98
Epoch 0011: Loss 24.6071 |  | Time 0.98
Epoch 0012: Loss 22.7349 |  | Time 0.96
Epoch 0013: Loss 21.0353 |  | Time 0.97
Epoch 0014: Loss 19.4021 |  | Time 0.96
Epoch 0015: Loss 17.8645 |  | Time 0.98
Epoch 0016: Loss 16.4100 |  | Time 0.96
Epoch 0017: Loss 15.0537 |  | Time 0.97
Epoch 0018: Loss 13.8795 |  | Time 0.98
Epoch 0019: Loss 12.8639 |  | Time 0.97
Epoch 0020: Loss 12.0131 |  | Time 0.98
Epoch 0021: Loss 11.3278 |  | Time 0.97
Epoch 0022: Loss 10.7669 |  | Time 0.97
Epoch 0023: Loss 10.3089 |  | Time 0.98
Epoch 0024: Loss 9.9141 |  | Time 0.98
E

In [23]:
print("ini f1: ", f1_score_for)
print("ini precision: ", precision_score)
print("ini recall: ", recall_score)

ini f1:  0.6043874784323392
ini precision:  tensor(0.5808)
ini recall:  tensor(0.5808)


In [106]:
print(train_durration)
print(predict_durration)

[250.85678052902222, 315.7898392677307, 317.1439673900604]
[4.691774845123291, 5.131696939468384, 4.681374549865723]


## OCGNN

In [17]:
def predict_ocgnn(label_test, ocgnn_compile, pyG_test):
    
    label_test = torch.tensor(label_test)
    test_node_features = torch.tensor(test_node_features)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    ocgnn_ip_pred_res, ocgnn_ip_score_res = ocgnn_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')
    
    unique_values, counts = torch.unique(ocgnn_ip_pred_res, return_counts=True)
    print(unique_values, counts)

    precision_pygod = eval_precision_at_k(label_test, ocgnn_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, ocgnn_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)

    return precision_pygod, recall_pygod, f1_score

In [20]:
def make_ocgnn_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features,
                        label_test):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)
    test_node_features = torch.tensor(test_node_features)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    ocgnn_model = OCGNN(hid_dim=14, num_layers=32, weight_decay=1, 
                    contamination=0.37, lr=0.004, epoch=100, gpu=-1, 
                    verbose=3)
    ocgnn_compile = ocgnn_model.fit(pyG_train, label_train)
    return ocgnn_compile, pyG_test


In [24]:
ocgnn_model, graph_test = make_ocgnn_model(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test)
precision_score, recall_score, f1_score= predict_ocgnn(label_test, ocgnn_model, graph_test)
print("ini f1: ", f1_score)
print("ini precision: ", precision_score)
print("ini recall: ", recall_score)

C:\Users\asus\AppData\Local\Temp\ipykernel_6956\3720414239.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)
C:\Users\asus\AppData\Local\Temp\ipykernel_6956\3720414239.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train)
C:\Users\asus\AppData\Local\Temp\ipykernel_6956\3720414239.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Epoch 0000: Loss 0.0000 | AUC 0.9909 | Recall 0.9309 | Precision 0.9309 | AP 0.9840 | F1 0.9309 | Time 0.60
Epoch 0001: Loss 0.0008 | AUC 0.6040 | Recall 0.4923 | Precision 0.4923 | AP 0.5459 | F1 0.4923 | Time 0.61
Epoch 0002: Loss 0.0004 | AUC 0.9208 | Recall 0.8668 | Precision 0.8668 | AP 0.8908 | F1 0.8668 | Time 0.56
Epoch 0003: Loss 0.0009 | AUC 0.1921 | Recall 0.0543 | Precision 0.0543 | AP 0.2340 | F1 0.0542 | Time 0.63
Epoch 0004: Loss 0.0006 | AUC 0.6685 | Recall 0.6476 | Precision 0.6476 | AP 0.6067 | F1 0.6477 | Time 0.85
Epoch 0005: Loss 0.0004 | AUC 0.8439 | Recall 0.6233 | Precision 0.6233 | AP 0.7514 | F1 0.6233 | Time 0.61
Epoch 0006: Loss 0.0004 | AUC 0.6882 | Recall 0.5555 | Precision 0.5555 | AP 0.6082 | F1 0.5555 | Time 0.60
Epoch 0007: Loss 0.0004 | AUC 0.4301 | Recall 0.3544 | Precision 0.3544 | AP 0.3205 | F1 0.3545 | Time 0.62
Epoch 0008: Loss 0.0004 | AUC 0.4698 | Recall 0.0006 | Precision 0.0006 | AP 0.3347 | F1 0.0000 | Time 0.63
Epoch 0009: Loss 0.0004 | AU

## GAE

In [25]:
def make_gae_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    gae_model = GAE(hid_dim=12, num_layers=12, weight_decay=3,
                contamination=0.37, lr=0.001, epoch=100, gpu=-1,
                verbose=3, recon_s=True, sigmoid_s=True)
    
    gae_compile = gae_model.fit(pyG_train, label_train)
    return gae_compile

In [26]:
def predict_gae(gae_compile, test_graph, test_node_features, label_test):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    gae_ip_pred_res, gae_ip_score_res = gae_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')
    f1_score_pygod = eval_f1(label_test, gae_ip_pred_res)
    precision = eval_precision_at_k(label_test, gae_ip_score_res)
    recall = eval_recall_at_k(label_test, gae_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 score pygod: ", f1_score_pygod)
    
    return f1_score_pygod, precision, recall, f1_score

In [27]:
gae_model = make_gae_model(train_graph, train_node_features, label_train)
f1_score_pygod, precision_score, recall_score, f1_score = predict_gae(gae_model, test_graph, test_node_features, label_test)

C:\Users\asus\AppData\Local\Temp\ipykernel_6956\1294941806.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)
C:\Users\asus\AppData\Local\Temp\ipykernel_6956\1294941806.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train)


Epoch 0000: Loss 0.3449 | AUC 0.9597 | Recall 0.8432 | Precision 0.8432 | AP 0.9354 | F1 0.8432 | Time 5.51
Epoch 0001: Loss 0.3343 | AUC 0.9597 | Recall 0.8432 | Precision 0.8432 | AP 0.9355 | F1 0.8432 | Time 4.22
Epoch 0002: Loss 0.3248 | AUC 0.9598 | Recall 0.8433 | Precision 0.8433 | AP 0.9355 | F1 0.8433 | Time 4.13
Epoch 0003: Loss 0.3162 | AUC 0.9598 | Recall 0.8433 | Precision 0.8433 | AP 0.9356 | F1 0.8433 | Time 4.51
Epoch 0004: Loss 0.3086 | AUC 0.9598 | Recall 0.8433 | Precision 0.8433 | AP 0.9356 | F1 0.8433 | Time 4.44
Epoch 0005: Loss 0.3017 | AUC 0.9599 | Recall 0.8432 | Precision 0.8432 | AP 0.9357 | F1 0.8432 | Time 4.26
Epoch 0006: Loss 0.2954 | AUC 0.9599 | Recall 0.8433 | Precision 0.8433 | AP 0.9357 | F1 0.8433 | Time 4.34
Epoch 0007: Loss 0.2899 | AUC 0.9599 | Recall 0.8433 | Precision 0.8433 | AP 0.9358 | F1 0.8433 | Time 4.61
Epoch 0008: Loss 0.2849 | AUC 0.9600 | Recall 0.8435 | Precision 0.8435 | AP 0.9358 | F1 0.8434 | Time 4.42
Epoch 0009: Loss 0.2805 | AU

C:\Users\asus\AppData\Local\Temp\ipykernel_6956\3791293239.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)
C:\Users\asus\AppData\Local\Temp\ipykernel_6956\3791293239.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_test = torch.tensor(label_test)


Test: Loss 0.0000 | AUC 0.9582 | Recall 0.8724 | Precision 0.8724 | AP 0.9498 | F1 0.8725 | Time 0.86
F1 score:  tensor(0.8724)
Precision:  tensor(0.8724)
Recall:  tensor(0.8724)
F1 score pygod:  0.852900032282363


## CONAD

In [23]:
def make_conad_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    conad_model = CONAD(hid_dim=10, num_layers=16, 
                        lr=0.001, weight_decay= 1, contamination=0.37,
                        epoch=100, gpu=-1,  
                        weight=1, dropout=0.2, verbose=3)
    conad_compile = conad_model.fit(pyG_train, label_train)

    return conad_compile

In [26]:
def predict_conad(conda_compile, test_graph, test_node_features, label_test):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features
    
    conad_ip_pred_res, conad_ip_score_res = conda_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear', )
    f1_score_pygod = eval_f1(label_test, conad_ip_pred_res)
    precision = eval_precision_at_k(label_test, conad_ip_score_res)
    recall = eval_recall_at_k(label_test, conad_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    unique_values, counts = torch.unique(conad_ip_pred_res, return_counts=True)
    print(unique_values, counts)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 score pygod: ", f1_score_pygod)
    return f1_score_pygod, precision, recall

In [27]:
conad_model = make_conad_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_conad(conad_model, test_graph, label_test,  test_node_features)

Epoch 0000: Loss 37.5968 | AUC 0.7805 | Recall 0.6532 | Precision 0.6532 | AP 0.6571 | F1 0.6532 | Time 8.91
Epoch 0001: Loss 37.5945 | AUC 0.7805 | Recall 0.6521 | Precision 0.6521 | AP 0.6570 | F1 0.6521 | Time 8.62
Epoch 0002: Loss 37.5923 | AUC 0.7808 | Recall 0.6532 | Precision 0.6532 | AP 0.6574 | F1 0.6532 | Time 6.91
Epoch 0003: Loss 37.5907 | AUC 0.7807 | Recall 0.6540 | Precision 0.6540 | AP 0.6573 | F1 0.6540 | Time 7.60
Epoch 0004: Loss 37.5878 | AUC 0.7810 | Recall 0.6538 | Precision 0.6538 | AP 0.6574 | F1 0.6538 | Time 6.55
Epoch 0005: Loss 37.5842 | AUC 0.7817 | Recall 0.6549 | Precision 0.6549 | AP 0.6580 | F1 0.6549 | Time 7.04
Epoch 0006: Loss 37.5851 | AUC 0.7819 | Recall 0.6555 | Precision 0.6555 | AP 0.6583 | F1 0.6555 | Time 5.68
Epoch 0007: Loss 37.5827 | AUC 0.7820 | Recall 0.6543 | Precision 0.6543 | AP 0.6582 | F1 0.6543 | Time 6.49
Epoch 0008: Loss 37.5815 | AUC 0.7821 | Recall 0.6555 | Precision 0.6555 | AP 0.6583 | F1 0.6555 | Time 6.45
Epoch 0009: Loss 37

## AnomalyDAE

In [12]:
def make_anomalydae_model(train_graph, train_node_features, 
                        label_train):

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features
    label_train = label_train

    anomalydae_model = AnomalyDAE(hid_dim=12, emb_dim=4, 
                        lr=0.001, contamination=0.37,
                        epoch=100, gpu=0,
                        weight=1, verbose=3)
    anomalydae_compile = anomalydae_model.fit(pyG_train, label_train)

    return anomalydae_compile

In [13]:
def predict_anomalydae(label_test, anomalydae_compile, pyG_test):

    test_node_features = torch.tensor(test_node_features)
    label_test = torch.tensor(label_test)
    
    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    anomalydae_ip_pred_res, anomalydae_ip_score_res = anomalydae_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear', )
    
    precision_pygod = eval_precision_at_k(label_test, anomalydae_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, anomalydae_ip_score_res)
    f1_score = 2*(precision_pygod*recall_pygod)/(precision_pygod+recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return f1_score, precision_pygod, recall_pygod

In [17]:

anomalydae_model = make_anomalydae_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_anomalydae(label_test, anomalydae_model, graph_test)

Epoch 0000: Loss 6808.8037 | AUC 0.8459 | Recall 0.6921 | Precision 0.6921 | AP 0.7182 | F1 0.6921 | Time 0.97
Epoch 0001: Loss 146.2048 | AUC 0.8468 | Recall 0.6967 | Precision 0.6967 | AP 0.7175 | F1 0.6967 | Time 0.68
Epoch 0002: Loss 146.1078 | AUC 0.8468 | Recall 0.6967 | Precision 0.6967 | AP 0.7175 | F1 0.6967 | Time 8.99
Epoch 0003: Loss 145.8769 | AUC 0.8468 | Recall 0.6967 | Precision 0.6967 | AP 0.7175 | F1 0.6967 | Time 8.63
Epoch 0004: Loss 145.5796 | AUC 0.8468 | Recall 0.6967 | Precision 0.6967 | AP 0.7176 | F1 0.6967 | Time 8.81
Epoch 0005: Loss 145.2428 | AUC 0.8469 | Recall 0.6967 | Precision 0.6967 | AP 0.7177 | F1 0.6967 | Time 8.70
Epoch 0006: Loss 144.8798 | AUC 0.8469 | Recall 0.6967 | Precision 0.6967 | AP 0.7178 | F1 0.6967 | Time 8.75
Epoch 0007: Loss 144.4981 | AUC 0.8467 | Recall 0.6967 | Precision 0.6967 | AP 0.7178 | F1 0.6967 | Time 8.73
Epoch 0008: Loss 144.1026 | AUC 0.8467 | Recall 0.6965 | Precision 0.6965 | AP 0.7179 | F1 0.6965 | Time 8.77
Epoch 000

In [208]:
def make_guide_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features,
                        label_test):

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features
    label_train = label_train

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    guide_model = GUIDE(hid_a=64, hid_s=4, num_layers=4,  
                             weight_decay=1,alpha=0.5, contamination=0.1, lr=0.001, epoch=100, gpu=0, 
                             graphlet_size=16, selected_motif=False,
                             verbose=3)
    guide_compile = guide_model.fit(pyG_train, label_train)

    return guide_compile, pyG_test

In [209]:
def predict_guide(label_test, guide_compile, pyG_test):
    guide_ip_pred_res, guide_ip_score_res = guide_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear', )
    f1_score_pygod = eval_f1(label_test, guide_ip_pred_res)
    precision = eval_precision_at_k(label_test, guide_ip_score_res)
    recall = eval_recall_at_k(label_test, guide_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    unique_values, counts = torch.unique(guide_ip_pred_res, return_counts=True)
    print(unique_values, counts)
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 score pygod: ", f1_score_pygod)
    return f1_score_pygod, precision, recall

In [210]:

guide_model, graph_test = make_guide_model(train_graph, train_node_features, label_train,
                                            test_graph, test_node_features, label_test)
precision_score, recall_score, f1_score = predict_guide(label_test, guide_model, graph_test)